In [2]:
import numpy as np
import pandas as pd
import re

## Preparing simulated data w/ randomness

In [3]:
# new simulated data with randomness
allScens_wRandom = pd.read_csv('../DSSG2023-Heating-Loads-Data/heatPumpCal/randomSim/results_simRandom.csv', usecols=lambda x: x not in ['Unnamed: 0'])
allScens_wRandom.head()

City  Exist_Fuel_Type    Scenario                 Census_Area ANCSA_Region  \
0  Adak                1        Base  Aleutians West Census Area        Aleut   
1  Adak                1  Small Load  Aleutians West Census Area        Aleut   
2  Adak                1  Large Load  Aleutians West Census Area        Aleut   
3  Adak                1    Low Elec  Aleutians West Census Area        Aleut   
4  Adak                1      No PCE  Aleutians West Census Area        Aleut   

               Util_Name     PCE        Sq_Ft  Capital_Cost  Elec_Use_Jan  \
0  Adak -TDX Residential  0.7597  1289.367468   5596.353187    430.689472   
1  Adak -TDX Residential  0.7597   773.620481   5596.353187    344.551577   
2  Adak -TDX Residential  0.7597  1934.051202   5596.353187    430.689472   
3  Adak -TDX Residential  0.7597  1289.367468   5596.353187    301.482630   
4  Adak -TDX Residential  0.7597  1289.367468   5596.353187    430.689472   

   ...            NPV  CO2_lbs_saved  CO2_driving_miles_saved  Fuel_Use_Chg  \
0  ...  131373.774254   13359.265488             14978.954856           0.0   
1  ...   15024.836341    2022.201412              2267.374931           0.0   
2  ...  152621.284397   15678.504599             17579.380603           0.0   
3  ...  200148.537559   21442.336962             24042.025187           0.0   
4  ...  223215.458792   23009.767347             25799.492243           0.0   

   Fuel_Price_Incremental  Elec_Use_Chg  Elec_Rate_Incremental  \
0                     NaN  -8906.176992               1.325889   
1                     NaN  -1348.134275               1.332778   
2                     NaN -10452.336399               1.325889   
3                     NaN -14294.891308               1.286420   
4                     NaN -15339.844898               1.325889   

   Elec_Rate_Avg_Base      Econ   Exist_Fuel  
0            1.122179  Economic  Electricity  
1            0.925189  Economic  Electricity  
2            1.122317  Economic  Electricity  
3            1.113844  Economic  Electricity  
4            1.337507  Economic  Electricity  

[5 rows x 32 columns]

In [4]:
# retreive montly temp 
all_avg_temp = allScens_wRandom['Avg_Temp']

recs = []

for i in range(len(all_avg_temp)):
     avg_temp = re.findall('\[(.*?)\]', all_avg_temp[i])
     avg_temp = list(map(float, avg_temp[0].split(',')))    
     recs.append(avg_temp)

In [5]:
all_avg_temp_df = pd.DataFrame(recs)
all_avg_temp_df.columns = 'Avg_Temp_' + (all_avg_temp_df.columns + 1).astype(str)
all_avg_temp_df.head()

Avg_Temp_1  Avg_Temp_2  Avg_Temp_3  Avg_Temp_4  Avg_Temp_5  Avg_Temp_6  \
0      26.706      27.833      29.256      29.997      31.818      35.787   
1      26.664      27.790      29.211      29.950      31.768      35.731   
2      34.680      36.144      37.992      38.954      41.319      46.473   
3      39.999      41.687      43.818      44.927      47.655      53.599   
4      32.216      33.576      35.293      36.186      38.383      43.171   

   Avg_Temp_7  Avg_Temp_8  Avg_Temp_9  Avg_Temp_10  Avg_Temp_11  Avg_Temp_12  
0      39.544      40.967      40.145       35.104       30.510       27.509  
1      39.483      40.903      40.082       35.049       30.463       27.466  
2      51.352      53.200      52.132       45.586       39.621       35.723  
3      59.227      61.358      60.127       52.577       45.696       41.201  
4      47.703      49.420      48.428       42.347       36.805       33.185

In [6]:
allScens_wRandomTemp = pd.concat([allScens_wRandom, all_avg_temp_df], axis=1)

In [7]:
allScens_wRandomTemp.columns

Index(['City', 'Exist_Fuel_Type', 'Scenario', 'Census_Area', 'ANCSA_Region',
       'Util_Name', 'PCE', 'Sq_Ft', 'Capital_Cost', 'Elec_Use_Jan',
       'Elec_Use_May', 'Rebate_dol', 'Fuel_Esc_Rate', 'Exist_Unit_Fuel_Cost',
       'Design_Heat_Load', 'Design_Heat_Temp', 'COP', 'Max_HP_Cap_Reached',
       'HP_Load_Frac', 'Avg_Temp', 'Freezing_days', 'IRR', 'NPV',
       'CO2_lbs_saved', 'CO2_driving_miles_saved', 'Fuel_Use_Chg',
       'Fuel_Price_Incremental', 'Elec_Use_Chg', 'Elec_Rate_Incremental',
       'Elec_Rate_Avg_Base', 'Econ', 'Exist_Fuel', 'Avg_Temp_1', 'Avg_Temp_2',
       'Avg_Temp_3', 'Avg_Temp_4', 'Avg_Temp_5', 'Avg_Temp_6', 'Avg_Temp_7',
       'Avg_Temp_8', 'Avg_Temp_9', 'Avg_Temp_10', 'Avg_Temp_11',
       'Avg_Temp_12'],
      dtype='object')

In [8]:
# replacing NA in Exist_Unit_Fuel_Cost with 0 
# Note: the NA comes from the fact that those rows are simulating for the electricity's scenario
allScens_wRandomTemp['Exist_Unit_Fuel_Cost'].fillna(0, inplace=True)

In [9]:
# export a final CSV for regression
allScens_wRandomTemp.to_csv('../Output/allScens_wRandomness_augmented.csv')

## Preparing simulated data w/ Alan Mitchell's original code and TMY3 data 

In [10]:
# load all simulated data
baseCase = pd.read_excel('../DSSG2023-Heating-Loads-Data/heatPumpCal/baseSim/results_baseCase.xlsx', usecols=lambda x: x not in ['Unnamed: 0'])
medRebate = pd.read_excel('../DSSG2023-Heating-Loads-Data/heatPumpCal/baseSim/results_medRebate.xlsx', usecols=lambda x: x not in ['Unnamed: 0'])
highRebate = pd.read_excel('../DSSG2023-Heating-Loads-Data/heatPumpCal/baseSim/results_highRebate.xlsx', usecols=lambda x: x not in ['Unnamed: 0'])
medFuelEsc = pd.read_excel('../DSSG2023-Heating-Loads-Data/heatPumpCal/baseSim/results_medFuelEsc.xlsx', usecols=lambda x: x not in ['Unnamed: 0'])
highFuelEsc = pd.read_excel('../DSSG2023-Heating-Loads-Data/heatPumpCal/baseSim/results_highFuelEsc.xlsx', usecols=lambda x: x not in ['Unnamed: 0'])

In [11]:
# re-create columns for rebate_dol and fuel_esc_rate
baseCase['rebate_dol'] = 0
baseCase['fuel_esc_rate'] = 0.03

medRebate['rebate_dol'] = 2000
medRebate['fuel_esc_rate'] = 0.03
highRebate['rebate_dol'] = 8000
highRebate['fuel_esc_rate'] = 0.03

medFuelEsc['rebate_dol'] = 0
medFuelEsc['fuel_esc_rate'] = 0.06
highFuelEsc['rebate_dol'] = 0
highFuelEsc['fuel_esc_rate'] = 0.12

In [12]:
# combine all simulated scenarios
allScens = pd.concat([baseCase, medRebate, highRebate, medFuelEsc, highFuelEsc])

In [13]:
len(allScens)

19745

## Getting city-specific data

In [14]:
city_info = pd.read_csv("../DSSG2023-Heating-Loads-Data/heatPumpCal/city.csv")

In [15]:
city_info.head()

ID       Name   Latitude   Longitude  ERHRegionID  WAPRegionID  \
0   1  Anchorage  61.151960 -149.864100            2            2   
1   2       Adak  51.848896 -176.635422            2            3   
2   3     Akutan  54.134945 -165.772446            2            3   
3   4  Allakaket  66.561386 -152.649445            4            4   
4   5     Angoon  57.496647 -134.578900            1            1   

   ImprovementCostLevel  FuelRefer  FuelCityID  Oil1Price  ...  \
0                     1          0         NaN       4.45  ...   
1                     5          0         NaN       7.30  ...   
2                     5          0         NaN       4.93  ...   
3                     5          0         NaN       7.00  ...   
4                     3          0         NaN       6.20  ...   

                 TMYname                                      ElecUtilities  \
0  ANCHORAGE INTL AP, AK  [('Chugach Electric South- Residential', 1), (...   
1           ADAK NAS, AK  [('Adak -TDX Residential', 677), ('Adak - TDX ...   
2       DUTCH HARBOR, AK  [('Akutan, City of - Residential', 9), ('Akuta...   
3      BETTLES FIELD, AK  [('Allakaket - APT - Residential', 249), ('All...   
4  SITKA JAPONSKI AP, AK  [('Inside Passage Electric Coop - Residential'...   

   GasPrice  aris_city             census_city                 census_area  \
0  0.988132  Anchorage  Anchorage municipality      Anchorage municipality   
1       NaN       Adak               Adak city  Aleutians West Census Area   
2       NaN     Akutan             Akutan city      Aleutians East Borough   
3       NaN  Allakaket          Allakaket city   Yukon-Koyukuk Census Area   
4       NaN     Angoon             Angoon city   Hoonah-Angoon Census Area   

                 ancsa_region                         railbelt    hub  \
0  Cook Inlet Regional (CIRI)                         Railbelt   True   
1                       Aleut  Affordable Energy Strategy Area  False   
2                       Aleut  Affordable Energy Strategy Area  False   
3                       Doyon  Affordable Energy Strategy Area  False   
4        Sealaska Corporation  Affordable Energy Strategy Area  False   

                                      avg_elec_usage  
0  [674.3176110825, 616.54369648225, 568.80201815...  
1  [471.936998147, 423.45531781, 460.81549341, 42...  
2  [500.99156233, 439.984315846, 456.510655517, 4...  
3  [338.509000744, 291.889556239, 276.804469551, ...  
4  [277.080558981, 257.757954545, 237.15561277, 2...  

[5 rows x 30 columns]

In [16]:
# seperate nested average electric usage columns
recs = []

all_avg_elec_usage = city_info['avg_elec_usage']

for i in range(len(all_avg_elec_usage)):
     usages = re.findall('\[(.*?)\]', all_avg_elec_usage[i])
     usages = list(map(float, usages[0].split(',')))    
     recs.append(usages)

avg_elec_usage_df = pd.DataFrame(recs)
avg_elec_usage_df.columns = 'avg_elec_usage' + (avg_elec_usage_df.columns + 1).astype(str)
avg_elec_usage_df.head()

avg_elec_usage1  avg_elec_usage2  avg_elec_usage3  avg_elec_usage4  \
0       674.317611       616.543696       568.802018       559.529060   
1       471.936998       423.455318       460.815493       424.294303   
2       500.991562       439.984316       456.510656       421.946347   
3       338.509001       291.889556       276.804470       259.698748   
4       277.080559       257.757955       237.155613       239.508557   

   avg_elec_usage5  avg_elec_usage6  avg_elec_usage7  avg_elec_usage8  \
0       502.688658       474.855272       467.944954       483.580908   
1       414.198262       408.021373       428.550250       419.713312   
2       426.966353       380.763389       392.608544       400.059090   
3       241.227592       241.877881       237.014486       240.391296   
4       232.813956       253.427645       264.282545       268.637115   

   avg_elec_usage9  avg_elec_usage10  avg_elec_usage11  avg_elec_usage12  
0       498.776902        532.835538        575.696936        610.274162  
1       409.077785        425.893820        443.339960        463.328994  
2       410.119915        434.168675        445.715397        472.657556  
3       250.339562        260.703968        289.122744        324.815682  
4       250.881906        238.540192        250.781436        260.080513

In [17]:
city_info2 = city_info[['Name', 'TMYid', 'Longitude', 'Latitude', 'Oil1Price', 'PropanePrice', 'GasPrice', 'railbelt']]
city_info_wElec = pd.concat([city_info2, avg_elec_usage_df], axis=1)
city_info_wElec.head()

Name   TMYid   Longitude   Latitude  Oil1Price  PropanePrice  \
0  Anchorage  702730 -149.864100  61.151960       4.45          6.48   
1       Adak  704540 -176.635422  51.848896       7.30           NaN   
2     Akutan  704890 -165.772446  54.134945       4.93           NaN   
3  Allakaket  701740 -152.649445  66.561386       7.00         12.92   
4     Angoon  703710 -134.578900  57.496647       6.20          6.23   

   GasPrice                         railbelt  avg_elec_usage1  \
0  0.988132                         Railbelt       674.317611   
1       NaN  Affordable Energy Strategy Area       471.936998   
2       NaN  Affordable Energy Strategy Area       500.991562   
3       NaN  Affordable Energy Strategy Area       338.509001   
4       NaN  Affordable Energy Strategy Area       277.080559   

   avg_elec_usage2  avg_elec_usage3  avg_elec_usage4  avg_elec_usage5  \
0       616.543696       568.802018       559.529060       502.688658   
1       423.455318       460.815493       424.294303       414.198262   
2       439.984316       456.510656       421.946347       426.966353   
3       291.889556       276.804470       259.698748       241.227592   
4       257.757955       237.155613       239.508557       232.813956   

   avg_elec_usage6  avg_elec_usage7  avg_elec_usage8  avg_elec_usage9  \
0       474.855272       467.944954       483.580908       498.776902   
1       408.021373       428.550250       419.713312       409.077785   
2       380.763389       392.608544       400.059090       410.119915   
3       241.877881       237.014486       240.391296       250.339562   
4       253.427645       264.282545       268.637115       250.881906   

   avg_elec_usage10  avg_elec_usage11  avg_elec_usage12  
0        532.835538        575.696936        610.274162  
1        425.893820        443.339960        463.328994  
2        434.168675        445.715397        472.657556  
3        260.703968        289.122744        324.815682  
4        238.540192        250.781436        260.080513

In [18]:
# Use Alan's function to get TMY data for each city
import requests
import urllib
import io

base_url = 'https://github.com/alanmitchell/akwlib-export/raw/main/data/v01/'

def get_df(file_path):
    """Returns a Pandas DataFrame that is found at the 'file_path'
    below the Base URL for accessing data.  The 'file_path' should end
    with '.pkl' and points to a pickled, compressed (bz2), Pandas DataFrame.
    """
    b = requests.get(urllib.parse.urljoin(base_url, file_path)).content
    df = pd.read_pickle(io.BytesIO(b), compression='bz2')
    return df
def tmy_from_id(tmy_id):
    """Returns a DataFrame of TMY data for the climate site identified
    by 'tmy_id'.
    """
    df = get_df(f'tmy3/{tmy_id}.pkl')
    return df

In [19]:
all_TMYid = city_info_wElec['TMYid']
unique_TMYid = all_TMYid.unique()

In [20]:
resc = []

for i in range(len(unique_TMYid)):
    TMYid = unique_TMYid[i]
    print(TMYid)
    tmy_data = tmy_from_id(TMYid)
    tmy_data['TMYid'] = TMYid
    resc.append(tmy_data)

702730
704540
704890
701740
703710
702320
701940
703810
700260
702190
702670
702746
702647
702740
702910
703330
702710
702600
703870
703160
702770
702960
700637
701043
701718
700197
701330
702000
701195
702070
702040
702035
702186
702084
702005
702185
703606
703407
703210
703260
703400
703165
703080
702075
702310
702225
702460
701780
702610
703620
703670
702607
703610
703855
703860
703950
703980
703884
702750
702756
702757
703410
702595
702590
702495
702510
703500
701625
702650


In [21]:
allTemp = pd.concat(resc)

In [22]:
monthlyTemp = allTemp.groupby(['month', 'TMYid'])['db_temp'].mean()
monthlyTemp = pd.DataFrame(monthlyTemp)

In [23]:
monthlyTemp.head()

db_temp
month TMYid            
1     700197 -12.390565
      700260 -10.765968
      700637 -11.686532
      701043   8.381290
      701195   5.037016

In [24]:
monthlyTemp_wide = pd.pivot_table(monthlyTemp, values='db_temp', index=['TMYid'], columns=['month'])
monthlyTemp_wide.columns = "avgTemp_" + monthlyTemp_wide.columns.astype(str)
monthlyTemp_wide.reset_index(inplace=True)

In [25]:
monthlyTemp_wide.head()

month   TMYid  avgTemp_1  avgTemp_2  avgTemp_3  avgTemp_4  avgTemp_5  \
0      700197 -12.390565  -0.454107  13.254355   20.53475  39.306210   
1      700260 -10.765968 -14.770268 -10.874355   -0.99850  20.421210   
2      700637 -11.686532 -23.615982 -17.823468    9.67825  29.109355   
3      701043   8.381290   1.973214   9.692339    8.85075  32.599758   
4      701195   5.037016  -3.676696   6.283952   19.37725  32.279194   

month  avgTemp_6  avgTemp_7  avgTemp_8  avgTemp_9  avgTemp_10  avgTemp_11  \
0       46.15375  54.510161  49.987419   37.47200   30.643468     9.64725   
1       33.28950  38.491371  39.575242   31.75075   14.321774    -4.77225   
2       41.87175  42.716774  40.105806   34.80750   23.679113    10.44375   
3       34.76975  41.486532  44.383226   36.26625   31.657903    19.23725   
4       38.16850  48.538710  46.906371   39.51325   33.439274    22.16325   

month  avgTemp_12  
0        7.615323  
1       -8.852258  
2       -3.129516  
3       17.553548  
4       13.454677

In [36]:
freezing_days = (allTemp.groupby([allTemp.index.day, 'month', 'TMYid'])['db_temp'].mean() < 32).groupby('TMYid').mean()
freezing_days.rename('freezing_days', inplace=True)
freezing_days = freezing_days.reset_index()

In [39]:
city_info_wTemp = city_info_wElec.merge(monthlyTemp_wide, on='TMYid', how="left").merge(freezing_days, on='TMYid', how="left")

In [40]:
city_info_wTemp.head()

Name   TMYid   Longitude   Latitude  Oil1Price  PropanePrice  \
0  Anchorage  702730 -149.864100  61.151960       4.45          6.48   
1       Adak  704540 -176.635422  51.848896       7.30           NaN   
2     Akutan  704890 -165.772446  54.134945       4.93           NaN   
3  Allakaket  701740 -152.649445  66.561386       7.00         12.92   
4     Angoon  703710 -134.578900  57.496647       6.20          6.23   

   GasPrice                         railbelt  avg_elec_usage1  \
0  0.988132                         Railbelt       674.317611   
1       NaN  Affordable Energy Strategy Area       471.936998   
2       NaN  Affordable Energy Strategy Area       500.991562   
3       NaN  Affordable Energy Strategy Area       338.509001   
4       NaN  Affordable Energy Strategy Area       277.080559   

   avg_elec_usage2  ...  avgTemp_4  avgTemp_5  avgTemp_6  avgTemp_7  \
0       616.543696  ...   37.86750  49.235968   56.61200  58.772097   
1       423.455318  ...   36.26575  38.467419   43.26575  47.808306   
2       439.984316  ...   36.62175  40.538629   45.16250  49.645806   
3       291.889556  ...   26.90000  46.350645   56.70550  61.006613   
4       257.757955  ...   39.40400  46.449113   52.21725  54.312258   

   avgTemp_8  avgTemp_9  avgTemp_10  avgTemp_11  avgTemp_12  freezing_days  
0  56.691452   51.30225   34.020161    22.06225   18.776532       0.402740  
1  49.528468   48.53475   42.440242    36.88650   33.257823       0.128767  
2  52.046774   47.48250   40.591613    39.23075   34.619919       0.153425  
3  53.408387   38.39250   19.701210     3.40250   -8.056532       0.545205  
4  55.614839   53.06500   45.399839    37.71300   31.990323       0.139726  

[5 rows x 33 columns]

In [42]:
# Some final processing
city_info_wTemp['Oil1Price'] = city_info_wTemp['Oil1Price'].fillna(0)
city_info_wTemp['PropanePrice'] = city_info_wTemp['PropanePrice'].fillna(0)
city_info_wTemp['GasPrice'] = city_info_wTemp['GasPrice'].fillna(0)
city_info_wTemp = pd.get_dummies(city_info_wTemp, columns=['railbelt'], drop_first =True)
city_info_wTemp.rename(columns={'railbelt_Railbelt':'Railbelt'}, inplace=True)
city_info_wTemp.rename(columns={'Name':'City'}, inplace=True)

KeyError: "None of [Index(['railbelt'], dtype='object')] are in the [columns]"

In [43]:
city_info_wTemp.to_csv('../Output/city_augmented.csv')

## Merging city info with the simulated data

In [44]:
city_augmented = pd.read_csv('../Output/city_augmented.csv', usecols=lambda x: x not in ['Unnamed: 0'])
allScens_augmented = allScens.merge(city_augmented, on='City')

In [45]:
allScens_augmented.head()

City  Exist_Fuel_Type    Scenario                 Census_Area ANCSA_Region  \
0  Adak                1        Base  Aleutians West Census Area        Aleut   
1  Adak                1  Small Load  Aleutians West Census Area        Aleut   
2  Adak                1  Large Load  Aleutians West Census Area        Aleut   
3  Adak                1    Low Elec  Aleutians West Census Area        Aleut   
4  Adak                1      No PCE  Aleutians West Census Area        Aleut   

               Util_Name     PCE   Sq_Ft  Capital_Cost  Elec_Use_Jan  ...  \
0  Adak -TDX Residential  0.7597  1130.0        6400.0    471.936998  ...   
1  Adak -TDX Residential  0.7597   678.0        6400.0    377.549599  ...   
2  Adak -TDX Residential  0.7597  1695.0        6400.0    471.936998  ...   
3  Adak -TDX Residential  0.7597  1130.0        6400.0    330.355899  ...   
4  Adak -TDX Residential  0.7597  1130.0        6400.0    471.936998  ...   

   avgTemp_5  avgTemp_6  avgTemp_7  avgTemp_8  avgTemp_9  avgTemp_10  \
0  38.467419   43.26575  47.808306  49.528468   48.53475   42.440242   
1  38.467419   43.26575  47.808306  49.528468   48.53475   42.440242   
2  38.467419   43.26575  47.808306  49.528468   48.53475   42.440242   
3  38.467419   43.26575  47.808306  49.528468   48.53475   42.440242   
4  38.467419   43.26575  47.808306  49.528468   48.53475   42.440242   

   avgTemp_11  avgTemp_12  freezing_days  Railbelt  
0     36.8865   33.257823       0.128767         0  
1     36.8865   33.257823       0.128767         0  
2     36.8865   33.257823       0.128767         0  
3     36.8865   33.257823       0.128767         0  
4     36.8865   33.257823       0.128767         0  

[5 rows x 60 columns]

In [46]:
allScens_augmented.to_csv('../Output/allScens_augmented.csv')